In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import oge.download_data as download_data
import oge.load_data as load_data
from oge.column_checks import get_dtypes, apply_dtypes, DATA_COLUMNS
from oge.filepaths import *
import oge.impute_hourly_profiles as impute_hourly_profiles
import oge.data_cleaning as data_cleaning
import oge.output_data as output_data
import oge.emissions as emissions
import oge.helpers as helpers
import oge.validation as validation
import oge.gross_to_net_generation as gross_to_net_generation
import oge.eia930 as eia930
from oge.logging_util import get_logger, configure_root_logger
import oge.constants as constants

configure_root_logger()
logger = get_logger("test")


year = 2024
path_prefix = f"{year}/"

In [5]:
# load intermediate output data as needed
cems = pd.read_csv(
    outputs_folder(f"{year}/cems_subplant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
partial_cems_subplant = pd.read_csv(
    outputs_folder(f"{year}/partial_cems_subplant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
partial_cems_plant = pd.read_csv(
    outputs_folder(f"{year}/partial_cems_plant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
eia923_allocated = pd.read_csv(
    outputs_folder(f"{year}/eia923_allocated_{year}.csv.zip"),
    compression="zip",
    parse_dates=["report_date"],
)
plant_attributes = pd.read_csv(
    outputs_folder(f"{year}/plant_static_attributes_{year}.csv.zip"), compression="zip"
)
primary_fuel_table = pd.read_csv(
    outputs_folder(f"{year}/primary_fuel_table_{year}.csv.zip"), compression="zip"
)
monthly_eia_data_to_shape = eia923_allocated[
    (eia923_allocated["hourly_data_source"] == "eia")
]

In [2]:
profiles = pd.read_csv(
    outputs_folder(f"{year}/hourly_profiles_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)

In [3]:
profiles[
    (profiles["ba_code"] == "SWPP")
    & (profiles["report_date"] == "2024-11-01")
    & (profiles["fuel_category"] == "petroleum")
]

,ba_code,fuel_category,datetime_utc,datetime_local,report_date,eia930_profile,residual_profile,scaled_residual_profile,shifted_residual_profile,imputed_profile,cems_profile,profile,profile_method,flat_profile
3588298,SWPP,petroleum,2024-11-01 05:00:00+00:00,2024-11-01 00:00:00-05:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3588299,SWPP,petroleum,2024-11-01 06:00:00+00:00,2024-11-01 01:00:00-05:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3588300,SWPP,petroleum,2024-11-01 07:00:00+00:00,2024-11-01 02:00:00-05:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3588301,SWPP,petroleum,2024-11-01 08:00:00+00:00,2024-11-01 03:00:00-05:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3588302,SWPP,petroleum,2024-11-01 09:00:00+00:00,2024-11-01 04:00:00-05:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589014,SWPP,petroleum,2024-12-01 01:00:00+00:00,2024-11-30 19:00:00-06:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3589015,SWPP,petroleum,2024-12-01 02:00:00+00:00,2024-11-30 20:00:00-06:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3589016,SWPP,petroleum,2024-12-01 03:00:00+00:00,2024-11-30 21:00:00-06:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0
3589017,SWPP,petroleum,2024-12-01 04:00:00+00:00,2024-11-30 22:00:00-06:00,2024-11-01,NaN,NaN,NaN,NaN,1.0,NaN,1.0,assumed_flat,1.0


In [4]:
px.line(
    profiles[
        (profiles["ba_code"] == "SWPP")
        & (profiles["report_date"] == "2024-11-01")
        & (profiles["fuel_category"] == "petroleum")
    ],
    x="datetime_utc",
    y="profile",
)

In [2]:
gens = load_data.load_pudl_table("out_eia__yearly_generators")

In [3]:
gens[gens["plant_id_eia"] == 55209]

,plant_id_eia,generator_id,report_date,unit_id_pudl,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,technology_description,energy_source_code_1,prime_mover_code,generator_operating_date,generator_retirement_date,operational_status,capacity_mw,fuel_type_code_pudl,planned_generator_retirement_date,capacity_factor,fuel_cost_per_mmbtu,fuel_cost_per_mwh,unit_heat_rate_mmbtu_per_mwh,net_generation_mwh,total_fuel_cost,total_mmbtu,associated_combined_heat_power,bga_source,bypass_heat_recovery,carbon_capture,city,county,current_planned_generator_operating_date,data_maturity,deliver_power_transgrid,distributed_generation,duct_burners,energy_source_1_transport_1,energy_source_1_transport_2,energy_source_1_transport_3,energy_source_2_transport_1,energy_source_2_transport_2,energy_source_2_transport_3,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,energy_storage_capacity_mwh,ferc_qualifying_facility,fluidized_bed_tech,fuel_type_count,latitude,longitude,minimum_load_mw,nameplate_power_factor,net_capacity_mwdc,operational_status_code,original_planned_generator_operating_date,other_combustion_tech,other_modifications_date,other_planned_modifications,owned_by_non_utility,ownership_code,planned_derate_date,planned_energy_source_code_1,planned_modifications,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_derate_mw,planned_net_winter_capacity_uprate_mw,planned_new_capacity_mw,planned_new_prime_mover_code,planned_repower_date,planned_uprate_date,previously_canceled,pulverized_coal_tech,reactive_power_output_mvar,rto_iso_lmp_node_id,rto_iso_location_wholesale_reporting_id,solid_fuel_gasification,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,state,stoker_tech,street_address,subcritical_tech,summer_capacity_estimate,summer_capacity_mw,summer_estimated_capability_mw,supercritical_tech,time_cold_shutdown_full_load_code,timezone,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,fuel_cost_per_mmbtu_source,can_cofire_fuels,can_burn_multiple_fuels,can_switch_oil_gas,synchronized_transmission_grid,can_switch_when_operating
411284,55209,GT4,2001-01-01,<NA>,7625,Brush 4,29116,1297,Colorado Energy Management LLC,Natural Gas Fired Combustion Turbine,NG,GT,1999-06-01,NaT,existing,35.000000,gas,NaT,0.112707,NaN,NaN,NaN,34556.000000,NaN,NaN,False,<NA>,<NA>,<NA>,Brush,Morgan,NaT,final,<NA>,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,<NA>,1,40.246895,-103.623199,NaN,NaN,NaN,SB,NaT,<NA>,NaT,<NA>,<NA>,S,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,<NA>,<NA>,NaN,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,CO,<NA>,1500 S. Clayton,<NA>,<NA>,11.0,NaN,<NA>,<NA>,America/Denver,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,False,12.0,NaN,80723,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
411285,55209,GT4,2002-01-01,<NA>,7625,Brush 4,29116,1297,Colorado Energy Management LLC,Natural Gas Fired Combustion Turbine,NG,GT,1999-06-01,NaT,existing,35.000000,gas,NaT,0.146133,NaN,NaN,NaN,44804.500000,NaN,NaN,False,<NA>,<NA>,<NA>,Brush,Morgan,NaT,final,<NA>,False,False,PL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,<NA>,1,40.246895,-103.623199,NaN,NaN,NaN,SB,NaT,<NA>,NaT,<NA>,<NA>,S,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,<NA>,<NA>,NaN,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,CO,<NA>,1500 S. Clayton,<NA>,<NA>,11.0,NaN,<NA>,<NA>,America/Denver,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,False,12.0,NaN,80723,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
411286,55209,GT4,2003-01-01,<NA>,7625,Brush 4,29116,1297,Colorado Energy Management LLC,Natural Gas Fired Combustion Turbine,NG,GT,1999-06-01,NaT,existing,55.000000,gas,NaT,0.077385,NaN,NaN,NaN,37284.000000,NaN,NaN,False,<NA>,<NA>,<NA>,Brush,Morgan,NaT,final,<NA>,False,False,PL,<NA>,<NA>,<NA>,<NA